# Theoretical Questions

* This is the theoretical part of the final project. It includes theoretical questions from various topics covered in the course.
* There are 7 questions among which you need to choose 6, according to the following key:
    + Question 1 is **mandatory**.
    + Choose **one question** from questions 2-3.
    + Question 4 is **mandatory**.
    + Questions 5-6 are **mandatory**.
    + Question 7 is **mandatory**.
* Question 1 is worth 15 points, whereas the other questions worth 7 points.
* All in all, the maximal grade for this parts is 15+7*5=50 points.
* **You should answer the questions on your own. We will check for plagiarism.**
* If you need to add external images (such as graphs) to this notebook, please put them inside the 'imgs' folder. DO NOT put a reference to an external link.
* Good luck!

## Part 1: General understanding of the course material

### Question 1

1.  Relate the number of parameters in a neural network to the over-fitting phenomenon (*).
    Relate this to the design of convolutional neural networks, and explain why CNNs are a plausible choice for an hypothesis class for visual classification tasks.

    (*) In the context of classical under-fitting/over-fitting in machine learning models.

2. Consider the linear classifier model with hand-crafted features: 
    $$f_{w,b}(x) = w^T \psi(x) + b$$
    where $x \in \mathbb{R}^2$, $\psi$ is a non-learnable feature extractor and assume that the classification is done by $sign(f_{w,b}(x))$. Let $\psi$ be the following feature extractor $\psi(x)=x^TQx$ where $Q \in \mathbb{R}^{2 \times 2}$ is a non-learnable positive definite matrix. Describe a distribution of the data which the model is able to approximate, but the simple linear model fails to approximate (hint: first, try to describe the decision boundary of the above classifier).

3. Assume that we would like to train a Neural Network for classifying images into $C$ classes. Assume that the architecture can be stored in the memory as a computational graph with $N$ nodes where the output is the logits (namely, before applying softmax) for the current batch ($f_w: B \times Ch \times H \times W \rightarrow B \times C$). Assume that the computational graph operates on *tensor* values.
    * Implement the CE loss assuming that the labels $y$ are hard labels given in a LongTensor (as usual). **Use Torch's log_softmax and index_select functions** and implement with less as possible operations.

In [ ]:
from torch.nn.functional import log_softmax
from torch import index_select
# Input:  model, x, y. 
# Output: the loss on the current batch.
logits = model(x)
...
loss = ...

* Using the model's function as a black box, draw the computational graph (treating both log_softmax and index_select as an atomic operations). How many nodes are there in the computational graph?

* Now, instead of using hard labels, assume that the labels are representing some probability distribution over the $C$ classes. How would the gradient computation be affected? analyze the growth in the computational graph, memory and computation.

* Apply the same analysis in the case that we would like to double the batch size. How should we change the learning rate of the optimizer?

## Part 2: Optimization & Automatic Differentiation

### Question 2: resolving gradient conflicts in multi-task learning

Assume that you want to train a model to perform two tasks: task 1 and task 2. 
For each such task $i$ you have an already implemented function *loss\_i = forward_and_compute_loss_i(model,inputs)* such that given the model and the inputs it computes the loss w.r.t task $i$ (assume that the computational graph is properly constructed). We would like to train our model using SGD to succeed in both tasks as follows: in each training iteration (batch) -
* Let $g_i$ be the gradient w.r.t the $i$-th task.
* If $g_1 \cdot g_2 < 0$:
    + Pick a task $i$ at random.
    + Apply GD w.r.t only that task.
* Otherwise:
    + Apply GD w.r.t both tasks (namely $\mathcal{L}_1 + \mathcal{L}_2$).

Note that in the above formulation the gradient is a thought of as a concatination of all the gradient w.r.t all the models parameters, and $g_1 \cdot g_2$ stands for a dot product.

What parts should be modified to implement the above? Is it the optimizer, the training loop or both? Implement the above algorithm in a code cell/s below

### Question 3: manual automatic differentiation

Consider the following two-input two-output function:
$$ f(x,y) = (x^2\sin(xy+\frac{\pi}{2}), x^2\ln(1+xy)) $$
* Draw a computational graph for the above function. Assume that the unary atomic units are squaring, taking square root, $\exp,\ln$, basic trigonometric functions and the binary atomic units are addition and multiplication. You would have to use constant nodes.
* Calculate manually the forward pass.
* Calculate manually the derivative of all outputs w.r.t all inputs using a forward mode AD.
* Calculate manually the derivative of all outputs w.r.t all inputs using a backward mode AD.

# Computational graph

<figure>
    <center><img src="imgs/computationgraph.jpg" /></center>
    <figcaption>
    computational graph
    </figcaption>
</figure>


## Illustatration of forward pass

<figure>
    <center><img src="imgs/forwardpass.jpg" /></center>
    <figcaption>
    forward pass
    </figcaption>
</figure>

# Forward pass:

$u_0 = x$

$u_{-1} = y$

$u_{-2} = 1$

$u_{-3} = \frac{\pi}{2}$

$u_1 = u_0 u_{-1} = xy$

$u_2 = u_1^2 = x^2$

$u_3 = u_1 + u_{-2} = xy + 1$

$u_4 = u_1 u_{-3} = xy + \frac{\pi}{2}$

$u_5 = ln(u_3) = ln(xy + 1)$

$u_6 = sin(u_4) = sin(xy + \frac{\pi}{2})$

$u_7 = u_2 u_5 = x^2 ln(xy + 1)$

$u_8 = u_2 u_6 = x^2 sin(xy + \frac{\pi}{2})$

$f_1 = u_8 = x^2 sin(xy + \frac{\pi}{2})$

$f_2 = u_7 = x^2 ln(xy + 1)$

# Forward mode
we have two inputs, t.f. we will have to compute the forward mode 2 times.

We will begin with x, which we denote as $u_0$

we will use $x$ with a single dot to notate partial derivation of $x$ w.r.t. $u_0$, i.e, $\dot{x} \triangleq  \frac{\partial{x}}{\partial{u_0}}$

$\dot{u_0} = 1$

$\dot{u_{-1}} = 0$

$\dot{u_{-2}} = 0$

$\dot{u_{-3}} = 0$

$\dot{u_{1}} = u_{-1} \dot{u_0} = u_{-1} = y$

$\dot{u_{2}} = 2u_{0} \dot{u_0} = 2 u_{0} = 2x$

$\dot{u_{3}} = \dot{u_1} = y$

$\dot{u_{4}} = \dot{u_{-3}} + \dot{u_1} = 0 + y = y$

$\dot{u_{5}} = \frac{1}{u_3}\dot{u_{3}} = frac{y}{1+xy}$

$\dot{u_{6}} = cos(u_4)\dot{u_{4}} = cos(xy + \frac{\pi}{2})(y)$

$\dot{u_{7}} = \dot{u_{5}}u_2 + \dot{u_{2}}u_5 =\frac{yx^2}{xy+1} + 2xln(xy+1)$

$\dot{u_{8}} = \dot{u_{6}}u_2 + \dot{u_{2}}u_6 = yx^2xcos(xy+\frac{\pi}{2}) + 2xsin(xy+\frac{\pi}{2})$



We now move on to y, which we denote as $u_{-1}$

We will use $x'$  to notate partial derivation of $x$ w.r.t. $u_{-1}$, i.e, $x' \triangleq  \frac{\partial{x}}{\partial{u_{-1}}}$

$u_{0}' = 0$

$u_{-1}' = 1$

$u_{-2}' = 0$

$u_{-3}' = 0$

$u_{1}' = u_0 = x$

$u_{2}' = 0$

$u_{3}' = u_{1}' = x$

$u_{4}' = u_{1}' = x$

$u_{5}' = \frac{1}{u_3} u_{3}' = \frac{x}{xy+1}$

$u_{6}' = cos(u_4)u_4' = xcos(xy+\frac{\pi}{2})$

$u_{7}' = f_2' = u_2u_{5}' = \frac{x^3}{xy+1}$

$u_{8}' = f_1' =  u_2u_{6}' = x^3cos(xy+\frac{\pi}{2})$


The other two constant nodes are constants, and as such deriving by them will result in $0$


# Backward mode:
we will use $x$ with a single bar to notate partial derivation of $f_2$ w.r.t. $x$, i.e, $\bar{x} \triangleq  \frac{\partial{f_2}}{\partial{x}}$

and $x$ with a double bar to notate partial derivation of $f_1$ w.r.t. $x$, i.e, $\bar{\bar{x}} \triangleq  \frac{\partial{f_1}}{\partial{x}}$


where $f_1$ and $f_2$ are the first and second outputs of $f$ respectively

# $f_1$

$\bar{\bar{u_8}} =  \frac{\partial{f_1}}{\partial{u_8}} = \frac{\partial{u_8}}{\partial{u_8}} = 1$

$\bar{\bar{u_2}} =  \frac{\partial{f_1}}{\partial{u_2}} = \bar{\bar{u_8}}\frac{\partial{u_8}}{\partial{u_2}} = 1 u_6 = u_6 = sin(xy + \frac{\pi}{2})$

$\bar{\bar{u_6}} =  \frac{\partial{f_1}}{\partial{u_6}} = \bar{\bar{u_8}}\frac{\partial{u_8}}{\partial{u_6}} = 1 u_2 = x^2$

$\bar{\bar{u_4}} =  \frac{\partial{f_1}}{\partial{u_4}} = \bar{\bar{u_6}}\frac{\partial{u_6}}{\partial{u_4}} = u_2 cos(u_4) = x^2 cos(xy + \frac{\pi}{2})$

$\bar{\bar{u_1}} =  \frac{\partial{f_1}}{\partial{u_1}} = \bar{\bar{u_4}}\frac{\partial{u_4}}{\partial{u_1}} = u_2 cos(u_4) 1 = u_2 cos(u_4) = x^2 cos(xy + \frac{\pi}{2})$

$\bar{\bar{u_0}} =  \frac{\partial{f_1}}{\partial{u_0}} = \bar{\bar{u_1}}\frac{\partial{u_1}}{\partial{u_0}} +  \bar{\bar{u_2}}\frac{\partial{u_2}}{\partial{u_0}} = u_2 cos(u_4) u_{-1} + u_6 2 u_0 = y x^2  cos(xy + \frac{\pi}{2}) + 2 x sin(xy + \frac{\pi}{2})$

$\bar{\bar{u_{-1}}} =  \frac{\partial{f_1}}{\partial{u_{-1}}} = \bar{\bar{u_1}}\frac{\partial{u_1}}{\partial{u_{-1}}} = u_2 cos(u_4) u_0 = u_2 u_0 cos(u_4) = x^2 x cos(xy + \frac{\pi}{2}) = x^3 cos(xy + \frac{\pi}{2})$

$\bar{\bar{u_{-3}}} =  \frac{\partial{f_1}}{\partial{u_{-3}}} = \bar{\bar{u_{4}}}\frac{\partial{u_4}}{\partial{u_{-3}}} = u_2 cos(u_4) 1 = u_2 cos(u_4) = x^2 cos(xy + \frac{\pi}{2})$

$\bar{\bar{u_{7}}} =  \frac{\partial{f_1}}{\partial{u_{7}}} = 0$

$\bar{\bar{u_{5}}} =  \frac{\partial{f_1}}{\partial{u_5}} = \bar{\bar{u_{7}}}  \frac{\partial{u_7}}{\partial{u_5}} = 0  \frac{\partial{u_7}}{\partial{u_5}} =0 $

$\bar{\bar{u_{3}}} =  \frac{\partial{f_1}}{\partial{u_3}} = \bar{\bar{u_{5}}}  \frac{\partial{u_5}}{\partial{u_3}} = 0  \frac{\partial{u_5}}{\partial{u_3}} =0 $

$\bar{\bar{u_{-2}}} =  \frac{\partial{f_1}}{\partial{u_{-2}}} = \bar{\bar{u_{3}}}  \frac{\partial{u_3}}{\partial{u_{-2}}} = 0  \frac{\partial{u_3}}{\partial{u_{-2}}} =0 $

# $f_2$

$\bar{u_7} =  \frac{\partial{f_2}}{\partial{u_7}} = \frac{\partial{u_7}}{\partial{u_7}} = 1$

$\bar{u_2} =  \frac{\partial{f_2}}{\partial{u_2}} = \bar{u_7}\frac{\partial{u_7}}{\partial{u_2}} = 1 u_5 = u_5 = ln(xy + 1)$

$\bar{u_5} =  \frac{\partial{f_2}}{\partial{u_5}} = \bar{u_7}\frac{\partial{u_7}}{\partial{u_5}} = 1 u_2 = u_2 = x^2$

$\bar{u_3} =  \frac{\partial{f_2}}{\partial{u_3}} = \bar{u_5}\frac{\partial{u_5}}{\partial{u_3}} = u_2 \frac{1}{u_3} = \frac{u_2}{u_3} = \frac{x^2}{xy + 1}$

$\bar{u_1} =  \frac{\partial{f_2}}{\partial{u_1}} = \bar{u_3}\frac{\partial{u_3}}{\partial{u_1}} = \bar{u_3} 1 = \frac{u_2}{u_3} = \frac{x^2}{xy + 1}$

$\bar{u_0} =  \frac{\partial{f_2}}{\partial{u_0}} = \bar{u_1}\frac{\partial{u_1}}{\partial{u_0}} +  \bar{u_2}\frac{\partial{u_2}}{\partial{u_0}} = \frac{u_2}{u_3} u_{-1} + u_5 2 u_0 = \frac{u_2 u_{-1}}{u_3} +  2 u_5  u_0= \frac{x^2 y}{xy + 1} + 2 x ln(xy + 1)$

$\bar{u_{-1}} =  \frac{\partial{f_2}}{\partial{u_{-1}}} = \bar{u_1}\frac{\partial{u_1}}{\partial{u_{-1}}} = \frac{u_2}{u_3} u_0 = \frac{u_2 u_0}{u_3} = \frac{x^2 x}{xy + 1} = \frac{x^3}{xy + 1}$

$\bar{u_{-2}} =  \frac{\partial{f_2}}{\partial{u_{-2}}} = \bar{u_3}\frac{\partial{u_3}}{\partial{u_{-2}}} = \frac{u_2}{u_3} 1 = \frac{u_2 }{u_3} = \frac{x^2}{xy + 1}$

$\bar{u_{8}} =  \frac{\partial{f_2}}{\partial{u_8}} = 0$

$\bar{u_{6}} =  \frac{\partial{f_2}}{\partial{u_6}} = \bar{u_{8}} \frac{\partial{u_8}}{\partial{u_6}} = 0  \frac{\partial{u_8}}{\partial{u_6}} =0 $

$\bar{u_{4}} =  \frac{\partial{f_2}}{\partial{u_4}} = \bar{u_{6}} \frac{\partial{u_6}}{\partial{u_4}} = 0  \frac{\partial{u_6}}{\partial{u_4}} =0 $

$\bar{u_{-3}} =  \frac{\partial{f_2}}{\partial{u_{-3}}} = \bar{u_{4}} \frac{\partial{u_4}}{\partial{u_{-3}}} = 0  \frac{\partial{u_4}}{\partial{u_{-3}}} =0 $

## Part 3: Sequential Models

### Question 4: RNNs vs Transformers in the real life

In each one of the following scenarios decide whether to use RNN based model or a transformer based model. Justify your choice.
1. You are running a start-up in the area of automatic summarization of academic papers. The inference of the model is done on the server side, and it is very important for it to be fast.
2. You need to design a mobile application that gathers small amount of data from few apps in every second and then uses a NN to possibly generate an alert given the information in the current second and the information from the past minute.
3. You have a prediction task over fixed length sequences on which you know the following properties:
    + In each sequence there are only few tokens that the model should attend to.
    + Most of the information needed for generating a reliable prediction is located at the beginning of the sequence.
    + There is no restriction on the computational resources.

## Part 4: Generative modeling

### Question 5: VAEs and GANS

Suggest a method for combining VAEs and GANs. Focus on the different components of the model and how to train them jointly (the objectives). Which drawbacks of these models the combined model may overcome? which not?

### Question 6: Diffusion Models

Show that $q(x_{t-1}|x_t,x_0)$ is tractable and is given by $\mathcal{N}(x_{t-1};\tilde{\mu}(x_t,x_0),\tilde{\beta_t}I)$ where the terms for $\tilde{\mu}(x_t,x_0)$ and $\tilde{\beta_t}$ are given in the last tutorial. Do so by explicitly computing the PDF.

## Part 5: Training Methods

### Question 7: Batch Normalization and Dropout

For both BatchNorm and Dropout analyze the following:
1. How to use them during the training phase (both in forward pass and backward pass)?
2. How differently they behave in the inference phase? How to distinguish these operation modes in code?
3. Assume you would like to perform multi-GPU training (*) to train your model. What should be done in order for BatchNorm and dropout to work properly? assume that each process holds its own copy of the model and that the processes can share information with each other.

(*): In a multi-GPU training each GPU is associated with its own process that holds an independent copy of the model. In each training iteration a (large) batch is split among these processes (GPUs) which compute the gradients of the loss w.r.t the relevant split of the data. Afterwards, the gradients from each process are then shared and averaged so that the GD would take into account the correct gradient and to assure synchornization of the model copies. Note that the proccesses are blocked between training iterations.